In [1]:
from flask import Flask, request, jsonify
from threading import Thread
import json
import os
from openai import OpenAI
import json
from vllm import SamplingParams
import json
from datetime import datetime, timezone, timedelta
from copy import deepcopy
import copy
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

INFO 07-13 08:09:34 [__init__.py:248] Automatically detected platform rocm.


In [2]:
app = Flask(__name__)
received_data = []

In [3]:
BASE_URL = f"http://localhost:3000/v1"
MODEL_PATH = "/home/user/Models/deepseek-ai/deepseek-llm-7b-chat"

In [4]:
client = OpenAI(api_key="abc-123", base_url=BASE_URL, timeout=None, max_retries=1)

In [5]:
from pydantic import BaseModel
class EmailMeetingInfo(BaseModel):
    Start: str
    End: str
    Duration_Min: int

In [20]:
class AI_AGENT:
    def __init__(self, client: OpenAI, MODEL_PATH: str):
        self.client = client
        self.model_path = MODEL_PATH

    def parse_email_and_location(self, email_text: str, location: str, curr_date:str) -> EmailMeetingInfo:
        dt = datetime.strptime(curr_date, "%d-%m-%YT%H:%M:%S")
        day_of_week = dt.strftime("%A")
        
        response = self.client.chat.completions.create(
            model=self.model_path,
            temperature=0.0,
            messages=[{
                "role": "user",
                "content": f"""
                You are an intelligent agent designed to assist in scheduling meetings.
                Given current datetime and day in GMT is : {curr_date} and {day_of_week}
                Your job is to extract and return the following information from the provided email text in raw json:
                
                1. Determine the meeting **date** from the email content.
                2. Use the given location: "{location}" to determine its **local timezone offset** (e.g., '+05:30', '-04:00', etc.).
                3. Set the **Start** time to the beginning of the given date in that location’s timezone: 'YYYY-MM-DDT00:00:00+OFFSET'.
                4. Set the **End** time to the end of that **next** date from the start date in that location’s timezone: 'YYYY-MM-DDT23:59:59+OFFSET'.
                5. Extract or deduce the meeting **duration in minutes** from the email (default range: 30 to 120 if not mentioned).
                7. If no specific date is mentioned, assume the meeting is scheduled for today **in the location’s timezone**.
                
                Assume all dates and times (except `datetime.now`) are in the **timezone of the provided location**.
                
                Return the output strictly as a JSON object with the following keys:
                - "Start" : string in ISO 8601 format using the location’s timezone of the meet from given datetime
                - "End" : string in ISO 8601 format using the location’s timezone of next date from given datetime
                - "Duration_Min" : integer value in minutes
                
                **Do not include any extra commentary or fields.**
                
                Email:
                {email_text}
                """
            }],
        )
        
        meeting = EmailMeetingInfo.model_validate_json(response.choices[0].message.content)
        return json.loads(meeting.model_dump_json())


In [21]:
def retrive_calendar_events(user, start, end):
    events_list = []
    token_path = "../Keys/"+user.split("@")[0]+".token"
    user_creds = Credentials.from_authorized_user_file(token_path)
    calendar_service = build("calendar", "v3", credentials=user_creds)
    events_result = calendar_service.events().list(calendarId='primary', timeMin=start,timeMax=end,singleEvents=True,orderBy='startTime').execute()
    events = events_result.get('items')
    
    for event in events : 
        attendee_list = []
        try:
            for attendee in event["attendees"]: 
                attendee_list.append(attendee['email'])
        except: 
            attendee_list.append("SELF")
        start_time = event["start"]
        end_time = event["end"]
        events_list.append(
            {"StartTime" : start_time, 
             "EndTime": end_time, 
             "NumAttendees" :len(set(attendee_list)), 
             "Attendees" : list(set(attendee_list)),
             "Summary" : event["summary"]})
    return events_list

In [22]:
ai_agent = AI_AGENT(client, MODEL_PATH)

In [23]:
def preprocess(sample_input):
    sample_input['Attendees'].append({"email":sample_input['From']})

def process_input(sample_input):
    preprocess(sample_input)
    res = ai_agent.parse_email_and_location(sample_input['EmailContent'], sample_input['Location'], sample_input['Datetime'])
    processed_input = {
        'Start': str(res['Start']),
        'End': str(res['End']),
        'Duration_mins': res['Duration_Min']
    }

    return deepcopy(sample_input) | processed_input

def get_output(processed_input):
    # deepcopy to avoid mutating nested lists like Attendees
    output = deepcopy(processed_input)

    for attendee in output['Attendees']:
        start = output['Start']
        end = output['End']
        events = retrive_calendar_events(attendee['email'], start, end)
        attendee['events'] = events

    return output

In [33]:
inp = {
    "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
    "Datetime": "09-07-2025T12:34:55",
    "Location": "IIT Mumbai",
    "From": "userone.amd@gmail.com",
    "Attendees": [
        {
            "email": "usertwo.amd@gmail.com"
        },
        {
            "email": "userthree.amd@gmail.com"
        }
    ],
    "Subject": "Agentic AI Project Status Update",
    "EmailContent": "Hi team, let's meet 15th July for 30 minutes to discuss the status of Agentic AI Project."
}
res = process_input(inp)

In [34]:
res

{'Request_id': '6118b54f-907b-4451-8d48-dd13d76033a5',
 'Datetime': '09-07-2025T12:34:55',
 'Location': 'IIT Mumbai',
 'From': 'userone.amd@gmail.com',
 'Attendees': [{'email': 'usertwo.amd@gmail.com'},
  {'email': 'userthree.amd@gmail.com'},
  {'email': 'userone.amd@gmail.com'}],
 'Subject': 'Agentic AI Project Status Update',
 'EmailContent': "Hi team, let's meet 15th July for 30 minutes to discuss the status of Agentic AI Project.",
 'Start': '2025-07-15T00:00:00+05:30',
 'End': '2025-07-15T23:59:59+05:30',
 'Duration_mins': 30}

In [13]:
get_output(res)

{'Request_id': '6118b54f-907b-4451-8d48-dd13d76033a5',
 'Datetime': '09-07-2025T12:34:55',
 'Location': 'IIT Mumbai',
 'From': 'userone.amd@gmail.com',
 'Attendees': [{'email': 'usertwo.amd@gmail.com',
   'events': [{'StartTime': {'dateTime': '2025-07-16T18:00:00+05:30',
      'timeZone': 'Asia/Kolkata'},
     'EndTime': {'dateTime': '2025-07-17T09:00:00+05:30',
      'timeZone': 'Asia/Kolkata'},
     'NumAttendees': 1,
     'Attendees': ['SELF'],
     'Summary': 'Off Hours'},
    {'StartTime': {'dateTime': '2025-07-17T18:00:00+05:30',
      'timeZone': 'Asia/Kolkata'},
     'EndTime': {'dateTime': '2025-07-18T09:00:00+05:30',
      'timeZone': 'Asia/Kolkata'},
     'NumAttendees': 1,
     'Attendees': ['SELF'],
     'Summary': 'Off Hours'},
    {'StartTime': {'dateTime': '2025-07-18T18:00:00+05:30',
      'timeZone': 'Asia/Kolkata'},
     'EndTime': {'dateTime': '2025-07-19T09:00:00+05:30',
      'timeZone': 'Asia/Kolkata'},
     'NumAttendees': 1,
     'Attendees': ['SELF'],
     'Su

In [14]:
import json
from datetime import datetime, timedelta
import pytz
from openai import OpenAI
from typing import List, Dict, Any, Tuple, Optional

class MeetingScheduler:
    def __init__(self, api_key: str, base_url: str):
        self.client = OpenAI(api_key=api_key, base_url=base_url, timeout=None, max_retries=0)
        self.metadata = []
        self.working_hours_start = 9  
        self.working_hours_end = 21   
        
    def parse_datetime(self, dt_str: str) -> datetime:
        """Parse datetime string to datetime object"""
        return datetime.fromisoformat(dt_str.replace('Z', '+00:00'))
    
    def format_datetime(self, dt: datetime) -> str:
        """Format datetime object to string"""
        return dt.strftime('%Y-%m-%dT%H:%M:%S%z')
    
    def is_within_working_hours(self, start_time: datetime, end_time: datetime) -> bool:
        """Check if the given time slot is within working hours (9 AM to 9 PM)"""
        return (start_time.hour >= self.working_hours_start and 
                end_time.hour <= self.working_hours_end and
                (end_time.hour < self.working_hours_end or end_time.minute == 0))
    
    def adjust_to_working_hours(self, dt: datetime) -> datetime:
        """Adjust datetime to fall within working hours"""
        if dt.hour < self.working_hours_start:
            return dt.replace(hour=self.working_hours_start, minute=0, second=0, microsecond=0)
        elif dt.hour >= self.working_hours_end:
            # Move to next day at 9 AM
            next_day = dt + timedelta(days=1)
            return next_day.replace(hour=self.working_hours_start, minute=0, second=0, microsecond=0)
        return dt
    
    def find_free_slots(self, attendees: List[Dict], start_time: datetime, end_time: datetime, duration_mins: int) -> List[Tuple[datetime, datetime]]:
        """Find free time slots for all attendees within the given timeframe and working hours"""
        timezone = pytz.timezone('Asia/Kolkata')
        
        # Adjust search window to working hours
        start_time = self.adjust_to_working_hours(start_time)
        
        # Create a list of all busy periods
        busy_periods = []
        
        for attendee in attendees:
            for event in attendee['events']:
                event_start = self.parse_datetime(event['StartTime']['dateTime'])
                event_end = self.parse_datetime(event['EndTime']['dateTime'])
                busy_periods.append((event_start, event_end))
        
        # Add working hours boundaries as busy periods
        current_date = start_time.date()
        end_date = end_time.date()
        
        while current_date <= end_date:
            # Add period before working hours (midnight to 9 AM)
            day_start = datetime.combine(current_date, datetime.min.time()).replace(tzinfo=start_time.tzinfo)
            working_start = day_start.replace(hour=self.working_hours_start)
            if day_start < working_start:
                busy_periods.append((day_start, working_start))
            
            # Add period after working hours (9 PM to midnight)
            working_end = day_start.replace(hour=self.working_hours_end)
            day_end = day_start.replace(hour=23, minute=59, second=59)
            if working_end < day_end:
                busy_periods.append((working_end, day_end))
            
            current_date += timedelta(days=1)
        
        # Sort busy periods by start time
        busy_periods.sort(key=lambda x: x[0])
        
        # Find free slots
        free_slots = []
        current_time = start_time
        
        for busy_start, busy_end in busy_periods:
            # Check if there's a gap before this busy period
            if current_time < busy_start:
                slot_duration = (busy_start - current_time).total_seconds() / 60
                if slot_duration >= duration_mins:
                    # Ensure the slot is within working hours
                    slot_end = current_time + timedelta(minutes=duration_mins)
                    if self.is_within_working_hours(current_time, slot_end):
                        free_slots.append((current_time, busy_start))
            
            # Update current time to after this busy period
            current_time = max(current_time, busy_end)
        
        # Check if there's time after the last busy period
        if current_time < end_time:
            slot_duration = (end_time - current_time).total_seconds() / 60
            if slot_duration >= duration_mins:
                slot_end = current_time + timedelta(minutes=duration_mins)
                if self.is_within_working_hours(current_time, slot_end):
                    free_slots.append((current_time, end_time))
        
        return free_slots
    
    def analyze_leisure_time(self, attendees: List[Dict]) -> List[Tuple[datetime, datetime, str, str]]:
        """Analyze events using NLP to find potential leisure/break times within working hours"""
        leisure_slots = []
        
        for attendee in attendees:
            email = attendee['email']
            for event in attendee['events']:
                event_summary = event['Summary']
                event_start = self.parse_datetime(event['StartTime']['dateTime'])
                event_end = self.parse_datetime(event['EndTime']['dateTime'])
                
                # Only consider events within working hours
                if not self.is_within_working_hours(event_start, event_end):
                    continue
                
                # Use NLP to analyze if this event represents leisure/break time
                nlp_analysis = self.analyze_event_summary(event_summary, event_start, event_end)
                
                if nlp_analysis['is_leisure']:
                    leisure_slots.append((event_start, event_end, email, nlp_analysis['reason']))
                    self.metadata.append(f"Identified leisure time for {email}: {event_summary} - {nlp_analysis['reason']}")
        
        return leisure_slots
    
    def analyze_event_summary(self, summary: str, start_time: datetime, end_time: datetime) -> Dict[str, Any]:
        """Use NLP to analyze event summary and determine if it's leisure time"""
        context = f"""
        Event Summary: "{summary}"
        Event Duration: {start_time} to {end_time}
        Event Length: {(end_time - start_time).total_seconds() / 3600:.1f} hours
        
        Analyze this event summary to determine if it represents leisure time, break time, or personal time where a work meeting could potentially be scheduled DURING WORKING HOURS (9 AM to 9 PM).
        
        Consider these factors:
        1. Keywords that indicate leisure: "off hours", "break", "lunch", "personal time", "free time", "rest", "relaxation"
        2. Keywords that indicate work: "meeting", "call", "presentation", "deadline", "project", "task", "urgent"
        3. Keywords that indicate unavailable: "travel", "appointment", "family", "doctor", "vacation", "sick"
        4. Time of day context (must be within 9 AM to 9 PM)
        5. Duration (very short events might not be suitable for meetings)
        
        Respond in JSON format with:
        {{
            "is_leisure": boolean,
            "confidence": float (0-1),
            "reason": "brief explanation",
            "suitability_score": float (0-1),
            "suggested_portion": "if applicable, suggest which part of the time slot would be best"
        }}
        """
        
        try:
            response = self.client.chat.completions.create(
                model=MODEL_PATH,
                messages=[
                    {"role": "system", "content": "You are an expert at analyzing calendar events to identify suitable meeting times within working hours (9 AM to 9 PM). Always respond with valid JSON."},
                    {"role": "user", "content": context}
                ],
                max_tokens=300,
                temperature=0.2
            )
            
            # Parse the JSON response
            analysis = json.loads(response.choices[0].message.content.strip())
            
            # Validate and set defaults
            analysis.setdefault('is_leisure', False)
            analysis.setdefault('confidence', 0.0)
            analysis.setdefault('reason', 'Unable to determine')
            analysis.setdefault('suitability_score', 0.0)
            analysis.setdefault('suggested_portion', None)
            
            return analysis
            
        except Exception as e:
            self.metadata.append(f"NLP Analysis Error for '{summary}': {str(e)}")
            # Fallback to simple keyword analysis
            return self.fallback_leisure_analysis(summary, start_time, end_time)
    
    def fallback_leisure_analysis(self, summary: str, start_time: datetime, end_time: datetime) -> Dict[str, Any]:
        """Fallback analysis when NLP fails"""
        summary_lower = summary.lower()
        
        leisure_keywords = ['off hours', 'break', 'lunch', 'personal', 'free', 'rest', 'leisure']
        work_keywords = ['meeting', 'call', 'presentation', 'deadline', 'project', 'urgent']
        unavailable_keywords = ['travel', 'appointment', 'family', 'doctor', 'vacation', 'sick']
        
        is_leisure = any(keyword in summary_lower for keyword in leisure_keywords)
        is_work = any(keyword in summary_lower for keyword in work_keywords)
        is_unavailable = any(keyword in summary_lower for keyword in unavailable_keywords)
        
        # Basic time-based logic - ensure within working hours
        hour = start_time.hour
        duration_hours = (end_time - start_time).total_seconds() / 3600
        
        # Check if within working hours
        if not self.is_within_working_hours(start_time, end_time):
            return {
                'is_leisure': False,
                'confidence': 0.9,
                'reason': 'Event is outside working hours (9 AM - 9 PM)',
                'suitability_score': 0.0
            }
        
        if is_unavailable:
            return {
                'is_leisure': False,
                'confidence': 0.8,
                'reason': 'Event indicates unavailability',
                'suitability_score': 0.0
            }
        elif is_work:
            return {
                'is_leisure': False,
                'confidence': 0.7,
                'reason': 'Event appears to be work-related',
                'suitability_score': 0.1
            }
        elif is_leisure or (duration_hours > 2 and self.working_hours_start <= hour <= self.working_hours_end):
            return {
                'is_leisure': True,
                'confidence': 0.6,
                'reason': 'Event appears to be leisure/personal time within working hours',
                'suitability_score': 0.7
            }
        else:
            return {
                'is_leisure': False,
                'confidence': 0.5,
                'reason': 'Unable to determine event type',
                'suitability_score': 0.3
            }
    
    def get_llm_decision(self, context: str, question: str) -> str:
        """Get LLM decision based on context and question"""
        try:
            response = self.client.chat.completions.create(
                model=MODEL_PATH,
                messages=[
                    {"role": "system", "content": "You are a meeting scheduling assistant. Ensure all meetings are scheduled within working hours (9 AM to 9 PM). Provide brief, practical advice."},
                    {"role": "user", "content": f"Context: {context}\n\nQuestion: {question}"}
                ],
                max_tokens=150,
                temperature=0.3
            )
            
            return response.choices[0].message.content.strip()
            
        except Exception as e:
            self.metadata.append(f"LLM Decision Error: {str(e)}")
            return "Unable to get LLM decision - proceeding with standard logic"
    
    def analyze_event_overlap(self, attendees: List[Dict], event_start: datetime, event_end: datetime) -> Dict[str, Any]:
        """Analyze potential conflicts when scheduling during existing events"""
        conflicts = []
        
        for attendee in attendees:
            email = attendee['email']
            for event in attendee['events']:
                existing_start = self.parse_datetime(event['StartTime']['dateTime'])
                existing_end = self.parse_datetime(event['EndTime']['dateTime'])
                
                # Check for overlap
                if (event_start < existing_end) and (event_end > existing_start):
                    conflicts.append({
                        'email': email,
                        'event_summary': event['Summary'],
                        'event_start': existing_start,
                        'event_end': existing_end,
                        'overlap_start': max(event_start, existing_start),
                        'overlap_end': min(event_end, existing_end)
                    })
        
        return {
            'conflicts': conflicts,
            'conflict_count': len(conflicts),
            'severity': 'high' if len(conflicts) > 1 else 'medium' if len(conflicts) == 1 else 'low'
        }
    
    def find_best_leisure_slot(self, leisure_slots: List[Tuple[datetime, datetime, str, str]], duration_mins: int) -> Optional[Tuple[datetime, datetime, str]]:
        """Find the best leisure slot using NLP analysis within working hours"""
        if not leisure_slots:
            return None
        
        # Get detailed analysis for each leisure slot
        analyzed_slots = []
        for start_time, end_time, email, reason in leisure_slots:
            slot_duration = (end_time - start_time).total_seconds() / 60
            
            # Ensure slot is within working hours and has adequate duration
            if slot_duration >= duration_mins and self.is_within_working_hours(start_time, end_time):
                # Ask LLM to evaluate this specific slot
                context = f"""
                Leisure slot analysis (within working hours 9 AM - 9 PM):
                - Email: {email}
                - Time: {start_time} to {end_time}
                - Duration: {slot_duration:.0f} minutes
                - Reason identified as leisure: {reason}
                - Meeting duration needed: {duration_mins} minutes
                
                Rate this slot for scheduling a team meeting on a scale of 1-10, considering:
                1. Time of day appropriateness (within 9 AM - 9 PM)
                2. Duration adequacy
                3. Likelihood of acceptance
                4. Professional appropriateness
                """
                
                try:
                    rating_response = self.client.chat.completions.create(
                        model=MODEL_PATH,
                        messages=[
                            {"role": "system", "content": "You are a scheduling expert. Rate the slot and provide a brief explanation. Ensure it's within working hours (9 AM - 9 PM)."},
                            {"role": "user", "content": context}
                        ],
                        max_tokens=150,
                        temperature=0.1
                    )
                    
                    rating_text = rating_response.choices[0].message.content.strip()
                    # Extract rating (simple approach)
                    rating = 5  # default
                    for word in rating_text.split():
                        if word.replace('.', '').isdigit():
                            rating = min(10, max(1, int(float(word))))
                            break
                    
                    analyzed_slots.append((start_time, end_time, email, rating, rating_text))
                    self.metadata.append(f"Slot rating for {email} at {start_time}: {rating}/10 - {rating_text}")
                    
                except Exception as e:
                    self.metadata.append(f"Rating error for leisure slot: {str(e)}")
                    analyzed_slots.append((start_time, end_time, email, 5, "Default rating due to error"))
        
        # Sort by rating and return the best slot
        if analyzed_slots:
            best_slot = max(analyzed_slots, key=lambda x: x[3])
            return (best_slot[0], best_slot[1], best_slot[2])
        
        return None
    
    def find_next_working_day_slot(self, start_time: datetime, duration_mins: int) -> Tuple[datetime, datetime]:
        """Find the next available slot in working hours, potentially on the next working day"""
        current_time = start_time
        
        # If current time is outside working hours, move to next working day
        if current_time.hour < self.working_hours_start:
            current_time = current_time.replace(hour=self.working_hours_start, minute=0, second=0, microsecond=0)
        elif current_time.hour >= self.working_hours_end:
            next_day = current_time + timedelta(days=1)
            current_time = next_day.replace(hour=self.working_hours_start, minute=0, second=0, microsecond=0)
        
        event_start = current_time
        event_end = event_start + timedelta(minutes=duration_mins)
        
        # Ensure the meeting ends within working hours
        if event_end.hour > self.working_hours_end or (event_end.hour == self.working_hours_end and event_end.minute > 0):
            # Move to next day if meeting would extend beyond working hours
            next_day = current_time + timedelta(days=1)
            event_start = next_day.replace(hour=self.working_hours_start, minute=0, second=0, microsecond=0)
            event_end = event_start + timedelta(minutes=duration_mins)
        
        return event_start, event_end
    
    def schedule_meeting(self, calendar_data: Dict) -> Dict:
        """Main function to schedule the meeting within working hours"""
        # Parse input data
        attendees = calendar_data['Attendees']
        start_time = self.parse_datetime(calendar_data['Start'])
        end_time = self.parse_datetime(calendar_data['End'])
        duration_mins = calendar_data['Duration_mins']
        subject = calendar_data['Subject']
        
        self.metadata.append(f"Starting meeting scheduling analysis for: {subject}")
        self.metadata.append(f"Requested duration: {duration_mins} minutes")
        self.metadata.append(f"Time window: {start_time} to {end_time}")
        self.metadata.append(f"Working hours constraint: {self.working_hours_start}:00 AM to {self.working_hours_end}:00 PM")
        
        # Step 1: Find free slots within working hours
        free_slots = self.find_free_slots(attendees, start_time, end_time, duration_mins)
        
        if free_slots:
            # Found free slots within working hours
            best_slot = free_slots[0]  # Take the first available slot
            event_start = best_slot[0]
            event_end = event_start + timedelta(minutes=duration_mins)
            
            # Double-check that the slot is within working hours
            if not self.is_within_working_hours(event_start, event_end):
                self.metadata.append("Selected slot is outside working hours, finding alternative...")
                event_start, event_end = self.find_next_working_day_slot(start_time, duration_mins)
            
            llm_context = f"Found {len(free_slots)} free slots within working hours. Selected slot: {event_start} to {event_end}"
            llm_decision = self.get_llm_decision(llm_context, "Is this a good time for a team meeting within working hours?")
            
            self.metadata.append(f"Found {len(free_slots)} free time slots within working hours")
            self.metadata.append(f"Selected slot: {event_start} to {event_end}")
            self.metadata.append(f"LLM Analysis: {llm_decision}")
            
        else:
            # No free slots found, check leisure time within working hours
            self.metadata.append("No completely free slots found within working hours. Analyzing leisure/break times using NLP...")
            
            leisure_slots = self.analyze_leisure_time(attendees)
            
            if leisure_slots:
                # Found potential leisure time within working hours - use NLP to find the best slot
                best_leisure_slot = self.find_best_leisure_slot(leisure_slots, duration_mins)
                
                if best_leisure_slot:
                    event_start = best_leisure_slot[0]
                    event_end = event_start + timedelta(minutes=duration_mins)
                    
                    # Ensure the meeting ends within working hours
                    if not self.is_within_working_hours(event_start, event_end):
                        self.metadata.append("Leisure slot extends beyond working hours, finding alternative...")
                        event_start, event_end = self.find_next_working_day_slot(start_time, duration_mins)
                    else:
                        # Analyze potential conflicts
                        conflict_analysis = self.analyze_event_overlap(attendees, event_start, event_end)
                        
                        llm_context = f"""
                        Selected leisure time slot analysis (within working hours):
                        - Time: {event_start} to {event_end}
                        - Attendee: {best_leisure_slot[2]}
                        - Conflicts found: {len(conflict_analysis['conflicts'])}
                        - Conflict details: {conflict_analysis['conflicts']}
                        """
                        
                        llm_decision = self.get_llm_decision(llm_context, "Should we schedule during this leisure time considering the conflicts?")
                        
                        self.metadata.append(f"Best leisure slot selected: {event_start} to {event_end}")
                        self.metadata.append(f"Conflict analysis: {len(conflict_analysis['conflicts'])} potential conflicts")
                        self.metadata.append(f"LLM Decision: {llm_decision}")
                    
                else:
                    # No suitable leisure slots found within working hours
                    self.metadata.append("No suitable leisure slots found within working hours after NLP analysis")
                    event_start, event_end = self.find_next_working_day_slot(start_time, duration_mins)
                    
                    llm_context = f"No suitable leisure slots found within working hours. Scheduling for next working day: {event_start} to {event_end}"
                    llm_decision = self.get_llm_decision(llm_context, "Is scheduling for the next working day appropriate?")
                    
                    self.metadata.append(f"Scheduling for next working day: {event_start} to {event_end}")
                    self.metadata.append(f"LLM Analysis: {llm_decision}")
                    
            else:
                # Schedule for next working day
                event_start, event_end = self.find_next_working_day_slot(start_time, duration_mins)
                
                llm_context = f"No leisure slots identified within working hours through NLP analysis. Scheduling for next working day: {event_start} to {event_end}"
                llm_decision = self.get_llm_decision(llm_context, "Is scheduling for the next working day appropriate?")
                
                self.metadata.append("No leisure slots identified within working hours through NLP analysis")
                self.metadata.append(f"Scheduling for next working day: {event_start} to {event_end}")
                self.metadata.append(f"LLM Analysis: {llm_decision}")
        
        # Final validation to ensure meeting is within working hours
        if not self.is_within_working_hours(event_start, event_end):
            self.metadata.append("Final validation failed - adjusting to working hours...")
            event_start, event_end = self.find_next_working_day_slot(start_time, duration_mins)
        
        # Create new event
        new_event = {
            'StartTime': {
                'dateTime': self.format_datetime(event_start),
                'timeZone': 'Asia/Kolkata'
            },
            'EndTime': {
                'dateTime': self.format_datetime(event_end),
                'timeZone': 'Asia/Kolkata'
            },
            'NumAttendees': len(attendees),
            'Attendees': [attendee['email'] for attendee in attendees],
            'Summary': subject
        }
        
        # Add event to each attendee's calendar
        for attendee in attendees:
            attendee['events'].append(new_event)
        
        # Update the calendar data
        calendar_data['EventStart'] = self.format_datetime(event_start)
        calendar_data['EventEnd'] = self.format_datetime(event_end)
        
        # Remove original Start and End keys
        del calendar_data['Start']
        del calendar_data['End']
        
        # Add metadata
        calendar_data['metadata'] = {
            'scheduling_analysis': self.metadata,
            'final_decision': f"Meeting scheduled from {event_start} to {event_end} (within working hours {self.working_hours_start}:00 AM - {self.working_hours_end}:00 PM)",
            'scheduling_strategy': self._get_scheduling_strategy(free_slots, leisure_slots if 'leisure_slots' in locals() else []),
            'working_hours_compliance': self.is_within_working_hours(event_start, event_end)
        }
        
        return calendar_data
    
    def _get_scheduling_strategy(self, free_slots: List, leisure_slots: List) -> str:
        """Determine which scheduling strategy was used"""
        if free_slots:
            return "free_slot_within_working_hours"
        elif leisure_slots:
            return "nlp_analyzed_leisure_time_within_working_hours"
        else:
            return "next_working_day"

In [15]:

def get_meeting_scheduled_by_ai(input_data):
    # Initialize scheduler
    processed_input = process_input(input_data)
    
    calender_data = get_output(processed_input)
    
    scheduler = MeetingScheduler(api_key="abc-123", base_url=BASE_URL)
    
    # Schedule the meeting
    updated_calendar = scheduler.schedule_meeting(calender_data)
    
    # Verify working hours compliance
    if updated_calendar['EventStart']:
        event_start = scheduler.parse_datetime(updated_calendar['EventStart'])
    if updated_calendar['EventEnd']:
        event_end = scheduler.parse_datetime(updated_calendar['EventEnd'])
    return updated_calendar

In [16]:
input_data = {
    "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
    "Datetime": "09-07-2025T12:34:55",
    "Location": "IIT Mumbai",
    "From": "userone.amd@gmail.com",
    "Attendees": [
        {
            "email": "usertwo.amd@gmail.com"
        },
        {
            "email": "userthree.amd@gmail.com"
        }
    ],
    "Subject": "Agentic AI Project Status Update",
    "EmailContent": "Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project."
}

In [17]:
print(json.dumps(get_meeting_scheduled_by_ai(input_data),indent=2))

{
  "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
  "Datetime": "09-07-2025T12:34:55",
  "Location": "IIT Mumbai",
  "From": "userone.amd@gmail.com",
  "Attendees": [
    {
      "email": "usertwo.amd@gmail.com",
      "events": [
        {
          "StartTime": {
            "dateTime": "2025-07-16T18:00:00+05:30",
            "timeZone": "Asia/Kolkata"
          },
          "EndTime": {
            "dateTime": "2025-07-17T09:00:00+05:30",
            "timeZone": "Asia/Kolkata"
          },
          "NumAttendees": 1,
          "Attendees": [
            "SELF"
          ],
          "Summary": "Off Hours"
        },
        {
          "StartTime": {
            "dateTime": "2025-07-17T18:00:00+05:30",
            "timeZone": "Asia/Kolkata"
          },
          "EndTime": {
            "dateTime": "2025-07-18T09:00:00+05:30",
            "timeZone": "Asia/Kolkata"
          },
          "NumAttendees": 1,
          "Attendees": [
            "SELF"
          ],
         

In [18]:
@app.route('/receive', methods=['POST'])
def receive():
    data = request.get_json()
    new_data = get_meeting_scheduled_by_ai(data)
    received_data.append(new_data)
    return jsonify(received_data)
    
@app.route('/health',methods=['GET'])
def func():
    return jsonify({"status":"healthy"}),200

def run_flask():
    app.run(host='0.0.0.0', port=5000)

In [19]:
# Start Flask in a background thread
Thread(target=run_flask, daemon=True).start()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://134.199.200.234:5000
Press CTRL+C to quit
152.58.2.17 - - [13/Jul/2025 08:10:11] "POST /receive HTTP/1.1" 200 -
